In [ ]:
# import everything
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard, EarlyStopping
import time


np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# load the data from numpy objects
X_train = np.load('../my_dataset/X_train.npy')
X_test = np.load('../my_dataset/X_test.npy')
X_validate = np.load('../my_dataset/X_validate.npy')

y_train = np.load('../my_dataset/y_train.npy')
y_test = np.load('../my_dataset/y_test.npy')
y_validate = np.load('../my_dataset/y_validate.npy')

In [ ]:
conv_layers = [3, 4]
conv_nodes = 48
epochs = 200
early_stops = 6
repeats = 5

dropouts = [ 0.1, 0.15, 0.2, 0.25, 0.3]
linear_dropouts = [0.1, 0.15, 0.2, 0.25, 0.3]

# equal dropout
for conv_layer in conv_layers:
    for dropout in dropouts:
        for i in range (repeats):
            NAME = "{}-conv-{}-convNodes-{}-equalDropout-{}".format(conv_layer, conv_nodes, dropout, i)
            print(NAME)

            model = Sequential()

            model.add(Conv2D(conv_nodes, (3, 3), input_shape=X_train.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(dropout))

            for l in range(conv_layer-1):
                model.add(Conv2D(conv_nodes, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
                model.add(Dropout(dropout))

            model.add(Flatten())

            # output layer (as big as the number of words we teach)
            output_neurons = len(np.unique(y_train))
            model.add(tf.keras.layers.Dense(output_neurons, activation=tf.nn.softmax))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='sparse_categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'],
                        )

            callbacks = [EarlyStopping(monitor='val_accuracy', patience=early_stops, restore_best_weights=True), tensorboard]
            model.fit(X_train, y_train,
                    epochs=epochs,
                    validation_data=(X_validate, y_validate),
                    callbacks=callbacks)

            # save the model
            model.save("models/"+NAME)


# linear dropout
for conv_layer in conv_layers:
    for dropout in dropouts:
        for i in range (repeats):
            NAME = "{}-conv-{}-convNodes-{}-linearDropout-{}".format(conv_layer, conv_nodes, dropout, i)
            print(NAME)

            model = Sequential()

            model.add(Conv2D(conv_nodes, (3, 3), input_shape=X_train.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(dropout))

            for l in range(conv_layer-1):
                model.add(Conv2D(conv_nodes, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
                model.add(Dropout(dropout/(l+1)))

            model.add(Flatten())

            # output layer (as big as the number of words we teach)
            output_neurons = len(np.unique(y_train))
            model.add(tf.keras.layers.Dense(output_neurons, activation=tf.nn.softmax))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='sparse_categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'],
                        )

            callbacks = [EarlyStopping(monitor='val_accuracy', patience=early_stops, restore_best_weights=True), tensorboard]
            model.fit(X_train, y_train,
                    epochs=epochs,
                    validation_data=(X_validate, y_validate),
                    callbacks=callbacks)

            # save the model
            model.save("models/"+NAME)

In [ ]:
averages = np.zeros((len(conv_layers), len(dropouts)))
X_validate = np.load('../my_dataset/X_validate.npy')
y_validate = np.load('../my_dataset/y_validate.npy')

conv_layers = [3, 4]
conv_nodes = 48
epochs = 200
early_stops = 6
repeats = 5

dropouts = [ 0.1, 0.15, 0.2, 0.25, 0.3]

for conv_layer in conv_layers:
    for dropout in dropouts:
        for i in range (repeats):
            NAME = "{}-conv-{}-convNodes-{}-equalDropout-{}".format(conv_layer, conv_nodes, dropout, i)
            model = tf.keras.models.load_model("models/"+NAME)
            accuracy = round(model.evaluate(X_validate, y_validate)[1] * 100, 2) 
            l = int(conv_layer-3)
            k = int((dropout-0.09)*20)
            averages[l][k] += accuracy
            print('| '+NAME+' | '+ str(accuracy)+' % |')

averages = averages/5
print(averages)

[91.0   91.000   91.622   92.372   91.500  ] <br>
[88.5   91.248   91.498   91.622   74.626]

#### 3 Convolutional nodes
* 3 Convolutional 0.1 vs 0.15 equal dropout

| 3-conv-48-convNodes-0.1-equalDropout   | Accuracy | - | 3-conv-48-convNodes-0.15-equalDropout   | Accuracy |
| -------------------------------------- | -------- | - | --------------------------------------- | -------  |
| 3-conv-48-convNodes-0.1-equalDropout-0 | 91.25 %  | - | 3-conv-48-convNodes-0.15-equalDropout-0 | 92.50 %  |
| 3-conv-48-convNodes-0.1-equalDropout-1 | 88.13 %  | - | 3-conv-48-convNodes-0.15-equalDropout-1 | 90.00 %  |
| 3-conv-48-convNodes-0.1-equalDropout-2 | 91.87 %  | - | 3-conv-48-convNodes-0.15-equalDropout-2 | 90.00 %  |
| 3-conv-48-convNodes-0.1-equalDropout-3 | 91.25 %  | - | 3-conv-48-convNodes-0.15-equalDropout-3 | 91.25 %  |
| 3-conv-48-convNodes-0.1-equalDropout-4 | 92.50 %  | - | 3-conv-48-convNodes-0.15-equalDropout-4 | 91.25 %  |
| Average                                | 91.00 %  | - | Average                                 | 91.00 %  |


* 3 Convolutional layers 0.2 vs 0.25 equal dropout

| 3-conv-48-convNodes-0.2-equalDropout   | Accuracy | - | 3-conv-48-convNodes-0.25-equalDropout   | Accuracy |
| -------------------------------------- | -------- | - | --------------------------------------  | -------- |
| 3-conv-48-convNodes-0.2-equalDropout-0 | 93.12 %  | - | 3-conv-48-convNodes-0.25-equalDropout-0 | 93.12 %  |
| 3-conv-48-convNodes-0.2-equalDropout-1 | 92.50 %  | - | 3-conv-48-convNodes-0.25-equalDropout-1 | 90.62 %  |
| 3-conv-48-convNodes-0.2-equalDropout-2 | 91.25 %  | - | 3-conv-48-convNodes-0.25-equalDropout-2 | 92.50 %  |
| 3-conv-48-convNodes-0.2-equalDropout-3 | 90.62 %  | - | 3-conv-48-convNodes-0.25-equalDropout-3 | 93.12 %  |
| 3-conv-48-convNodes-0.2-equalDropout-4 | 90.62 %  | - | 3-conv-48-convNodes-0.25-equalDropout-4 | 92.50 %  |
| 🟢 Average (2nd)                       | 91.62 %  | - | 🟢 Average (1st)                       | 92.37 %  |


* 3 Convolytional layers 48 nodes 0.3 equal dropout

| 3-conv-48-convNodes-0.3-equalDropout   | Accuracy |
| -------------------------------------- | -------- |
| 3-conv-48-convNodes-0.3-equalDropout-0 | 92.50 %  |
| 3-conv-48-convNodes-0.3-equalDropout-1 | 92.50 %  |
| 3-conv-48-convNodes-0.3-equalDropout-2 | 88.75 %  |
| 3-conv-48-convNodes-0.3-equalDropout-3 | 92.50 %  |
| 3-conv-48-convNodes-0.3-equalDropout-4 | 91.25 %  |
| 🟢 Average (3rd)                      | 91.50 %  |


#### 4 Convolutional nodes
* 4 Convolutional 0.1 vs 0.15 equal dropout

| 4-conv-48-convNodes-0.1-equalDropout   | Accuracy | - | 4-conv-48-convNodes-0.15-equalDropout   | Accuracy |
| -------------------------------------- | -------- | - | --------------------------------------- | -------  |
| 4-conv-48-convNodes-0.1-equalDropout-0 | 90.00 %  | - | 4-conv-48-convNodes-0.15-equalDropout-0 | 94.38 %  |
| 4-conv-48-convNodes-0.1-equalDropout-1 | 91.87 %  | - | 4-conv-48-convNodes-0.15-equalDropout-1 | 90.62 %  |
| 4-conv-48-convNodes-0.1-equalDropout-2 | 89.38 %  | - | 4-conv-48-convNodes-0.15-equalDropout-2 | 90.62 %  |
| 4-conv-48-convNodes-0.1-equalDropout-3 | 86.25 %  | - | 4-conv-48-convNodes-0.15-equalDropout-3 | 90.62 %  |
| 4-conv-48-convNodes-0.1-equalDropout-4 | 85.00 %  | - | 4-conv-48-convNodes-0.15-equalDropout-4 | 90.00 %  |
| Average                                | 88.50 %  | - | Average                                 | 91.24 %  |


* 4 Convolutional layers 0.2 vs 0.25 equal dropout

| 4-conv-48-convNodes-0.2-equalDropout   | Accuracy | - | 4-conv-48-convNodes-0.25-equalDropout   | Accuracy |
| -------------------------------------- | -------- | - | --------------------------------------  | -------- |
| 4-conv-48-convNodes-0.2-equalDropout-0 | 91.87 %  | - | 4-conv-48-convNodes-0.25-equalDropout-0 | 90.00 %  |
| 4-conv-48-convNodes-0.2-equalDropout-1 | 89.38 %  | - | 4-conv-48-convNodes-0.25-equalDropout-1 | 93.12 %  |
| 4-conv-48-convNodes-0.2-equalDropout-2 | 91.87 %  | - | 4-conv-48-convNodes-0.25-equalDropout-2 | 91.87 %  |
| 4-conv-48-convNodes-0.2-equalDropout-3 | 93.12 %  | - | 4-conv-48-convNodes-0.25-equalDropout-3 | 91.87 %  |
| 4-conv-48-convNodes-0.2-equalDropout-4 | 91.25 %  | - | 4-conv-48-convNodes-0.25-equalDropout-4 | 91.25 %  |
| 🟢 Average (4th)                       | 91.49 %  | - | Average                                 | 91.24 %  |


* 4 Convolytional layers 48 nodes 0.3 equal dropout

| 4-conv-48-convNodes-0.3-equalDropout   | Accuracy |
| -------------------------------------- | -------- |
| 4-conv-48-convNodes-0.3-equalDropout-0 | 94.38 %  |
| 4-conv-48-convNodes-0.3-equalDropout-1 | 6.25 %   |
| 4-conv-48-convNodes-0.3-equalDropout-2 | 88.13 %  |
| 4-conv-48-convNodes-0.3-equalDropout-3 | 93.75 %  |
| 4-conv-48-convNodes-0.3-equalDropout-4 | 90.62 %  |
| Average                                | 74.62 %  |

In [ ]:
averages = np.zeros((len(conv_layers), len(dropouts)))
X_validate = np.load('../my_dataset/X_validate.npy')
y_validate = np.load('../my_dataset/y_validate.npy')

conv_layers = [3, 4]
conv_nodes = 48
epochs = 200
early_stops = 6
repeats = 5

dropouts = [ 0.1, 0.15, 0.2, 0.25, 0.3]

for conv_layer in conv_layers:
    for dropout in dropouts:
        for i in range (repeats):
            NAME = "{}-conv-{}-convNodes-{}-linearDropout-{}".format(conv_layer, conv_nodes, dropout, i)
            model = tf.keras.models.load_model("models/"+NAME)
            accuracy = round(model.evaluate(X_validate, y_validate)[1] * 100, 2) 
            l = int(conv_layer-3)
            k = int((dropout-0.09)*20)
            averages[l][k] += accuracy
            print('| '+NAME+' | '+ str(accuracy)+' % |')

averages = averages/5
print(averages)

[90.248 90.872 91.374 90.876 92.122] <br>
[88.126 89.250 89.126 91.122 90.378]

#### 3 Convolutional nodes
* 3 Convolutional 0.1 vs 0.15 linear dropout

| 3-conv-48-convNodes-0.1-linearDropout   | Accuracy | - | 3-conv-48-convNodes-0.15-linearDropout   | Accuracy |
| --------------------------------------- | -------- | - | ---------------------------------------  | -------- |
| 3-conv-48-convNodes-0.1-linearDropout-0 | 90.62 %  | - | 3-conv-48-convNodes-0.15-linearDropout-0 | 88.75 %  |
| 3-conv-48-convNodes-0.1-linearDropout-1 | 87.50 %  | - | 3-conv-48-convNodes-0.15-linearDropout-1 | 91.25 %  |
| 3-conv-48-convNodes-0.1-linearDropout-2 | 91.87 %  | - | 3-conv-48-convNodes-0.15-linearDropout-2 | 91.87 %  |
| 3-conv-48-convNodes-0.1-linearDropout-3 | 91.87 %  | - | 3-conv-48-convNodes-0.15-linearDropout-3 | 90.62 %  |
| 3-conv-48-convNodes-0.1-linearDropout-4 | 89.38 %  | - | 3-conv-48-convNodes-0.15-linearDropout-4 | 91.87 %  |
| Average                                 | 90.24 %  | - | Average                                  | 90.87 %  |


* 3 Convolutional 0.2 vs 0.25 linear dropout

| 3-conv-48-convNodes-0.2-linearDropout   | Accuracy | - | 3-conv-48-convNodes-0.25-linearDropout   | Accuracy |
| --------------------------------------- | -------- | - | ---------------------------------------  | -------- |
| 3-conv-48-convNodes-0.2-linearDropout-0 | 90.00 %  | - | 3-conv-48-convNodes-0.25-linearDropout-0 | 90.00 %  |
| 3-conv-48-convNodes-0.2-linearDropout-1 | 91.87 %  | - | 3-conv-48-convNodes-0.25-linearDropout-1 | 91.25 %  |
| 3-conv-48-convNodes-0.2-linearDropout-2 | 92.50 %  | - | 3-conv-48-convNodes-0.25-linearDropout-2 | 91.25 %  |
| 3-conv-48-convNodes-0.2-linearDropout-3 | 90.00 %  | - | 3-conv-48-convNodes-0.25-linearDropout-3 | 89.38 %  |
| 3-conv-48-convNodes-0.2-linearDropout-4 | 92.50 %  | - | 3-conv-48-convNodes-0.25-linearDropout-4 | 92.50 %  |
| 🟢 Average (2nd)                       | 91.37 %  | - | Average                                  | 90.87 %  |


* 3 Convolutional layers 48 nodes 0.3 linear dropout

| 3-conv-48-convNodes-0.3-linearDropout   | Accuracy |
| --------------------------------------- | -------- |
| 3-conv-48-convNodes-0.3-linearDropout-0 | 93.75 %  |
| 3-conv-48-convNodes-0.3-linearDropout-1 | 93.12 %  |
| 3-conv-48-convNodes-0.3-linearDropout-2 | 90.62 %  |
| 3-conv-48-convNodes-0.3-linearDropout-3 | 90.00 %  |
| 3-conv-48-convNodes-0.3-linearDropout-4 | 93.12 %  |
| 🟢 Average (1st)                       | 92.12 %  |

#### 4 Convolutional nodes
* 4 Convolutional 0.1 vs 0.15 linear dropout

| 4-conv-48-convNodes-0.1-linearDropout   | Accuracy | - | 4-conv-48-convNodes-0.15-linearDropout   | Accuracy |
| --------------------------------------- | -------- | - | ---------------------------------------  | -------- |
| 4-conv-48-convNodes-0.1-linearDropout-0 | 91.25 %  | - | 4-conv-48-convNodes-0.15-linearDropout-0 | 85.00 %  |
| 4-conv-48-convNodes-0.1-linearDropout-1 | 89.38 %  | - | 4-conv-48-convNodes-0.15-linearDropout-1 | 90.00 %  |
| 4-conv-48-convNodes-0.1-linearDropout-2 | 85.00 %  | - | 4-conv-48-convNodes-0.15-linearDropout-2 | 92.50 %  |
| 4-conv-48-convNodes-0.1-linearDropout-3 | 87.50 %  | - | 4-conv-48-convNodes-0.15-linearDropout-3 | 90.62 %  |
| 4-conv-48-convNodes-0.1-linearDropout-4 | 87.50 %  | - | 4-conv-48-convNodes-0.15-linearDropout-4 | 88.13 %  |
| Average                                 | 88.12 %  | - | Average                                  | 89.25 %  |


* 4 Convolutional 0.2 vs 0.25 linear dropout

| 4-conv-48-convNodes-0.2-linearDropout   | Accuracy | - | 4-conv-48-convNodes-0.25-linearDropout   | Accuracy |
| --------------------------------------- | -------- | - | ---------------------------------------  | -------- |
| 4-conv-48-convNodes-0.2-linearDropout-0 | 88.13 %  | - | 4-conv-48-convNodes-0.25-linearDropout-0 | 90.62 %  |
| 4-conv-48-convNodes-0.2-linearDropout-1 | 86.25 %  | - | 4-conv-48-convNodes-0.25-linearDropout-1 | 91.87 %  |
| 4-conv-48-convNodes-0.2-linearDropout-2 | 90.00 %  | - | 4-conv-48-convNodes-0.25-linearDropout-2 | 90.00 %  |
| 4-conv-48-convNodes-0.2-linearDropout-3 | 90.00 %  | - | 4-conv-48-convNodes-0.25-linearDropout-3 | 91.87 %  |
| 4-conv-48-convNodes-0.2-linearDropout-4 | 91.25 %  | - | 4-conv-48-convNodes-0.25-linearDropout-4 | 91.25 %  |
| Average                                 | 89.12 %  | - | 🟢 Average (3rd)                        | 91.12 %  |

* 4 Convolutional 0.3 linear dropout

| 4-conv-48-convNodes-0.3-linearDropout   | Accuracy |
| --------------------------------------- | -------- |
| 4-conv-48-convNodes-0.3-linearDropout-0 | 94.38 %  |
| 4-conv-48-convNodes-0.3-linearDropout-1 | 88.75 %  |
| 4-conv-48-convNodes-0.3-linearDropout-2 | 90.00 %  |
| 4-conv-48-convNodes-0.3-linearDropout-3 | 89.38 %  |
| 4-conv-48-convNodes-0.3-linearDropout-4 | 89.38 %  |
| Average                                 | 90.37 %  |